In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import gc
import math
from joblib import Parallel, delayed

from tqdm import tqdm
import time
import warnings

warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
def get_psi(c):
    psi_res = pd.DataFrame()
    psi_dict={}
    # for c in tqdm(f_cols):
    try:
        t_train = x_train[c].fillna(-998)
        t_test = x_test[c].fillna(-998)
        #获取切分点
        bins=[]
        for i in np.arange(0,1.1,0.2):
            bins.append(t_train.quantile(i))
        bins=sorted(set(bins))
        bins[0]=-np.inf
        bins[-1]=np.inf
        #计算psi
        t_psi = pd.DataFrame()
        t_psi['train'] = pd.cut(t_train,bins).value_counts().sort_index()
        t_psi['test'] = pd.cut(t_test,bins).value_counts()
        t_psi.index=[str(x) for x in t_psi.index]
        t_psi.loc['总计',:] = t_psi.sum()
        t_psi['train_rate'] = t_psi['train']/t_psi.loc['总计','train']
        t_psi['test_rate'] = t_psi['test']/t_psi.loc['总计','test']
        t_psi['psi'] = (t_psi['test_rate']-t_psi['train_rate'])*(np.log(t_psi['test_rate'])-np.log(t_psi['train_rate']))
        t_psi.loc['总计','psi'] = t_psi['psi'].sum()
        t_psi.index.name=c
        #汇总
        t_res = pd.DataFrame([[c,t_psi.loc['总计','psi']]],
                             columns=['变量名','PSI'])
        psi_res = pd.concat([psi_res,t_res])
        psi_dict[c]=t_psi
        print(c,'done')
    except:
        print(c,'error')
    return psi_res #, psi_dict

In [3]:
def correlation(df, threshold=0.98):
    """
    特征相关性计算
    @param df: 
    @param threshold: 
    @return: 
    """
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colName = corr_matrix.columns[i]
                col_corr.add(colName)
    return col_corr

In [4]:
train_df = pd.read_hdf('../../input/train.h5')
test_df = pd.read_hdf('../../input/test.h5')

In [5]:
train_df = train_df[train_df['temperature'].notnull()]
train_df = train_df.fillna(method='bfill')
test_df = test_df.fillna(method='bfill')
gc.collect()

64

In [6]:
train_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                    'indoorHum', 'indoorAtmo', 'temperature']
test_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                   'indoorHum', 'indoorAtmo']

In [7]:
train_df.drop(['year', 'sec'], axis=1, inplace=True)
test_df.drop(['year', 'sec'], axis=1, inplace=True)

In [8]:
print('train_df.shape: ', train_df.shape)
train_df = train_df.loc[(train_df['outdoorTemp'] >= test_df['outdoorTemp'].min()) & (train_df['outdoorTemp'] <= test_df['outdoorTemp'].max())]
print('处理后 train_df.shape: ', train_df.shape)
gc.collect()

train_df.shape:  (24807, 11)
处理后 train_df.shape:  (19338, 11)


60

In [9]:
train_count = train_df.shape[0]
y_train = train_df['temperature'].values - train_df['outdoorTemp'].values

In [10]:
data_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

del train_df, test_df
gc.collect()

40

In [11]:
numerical_features = ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']
diff_features = ['{}_diff'.format(i) for i in numerical_features]
numerical_diff_features = numerical_features + diff_features

In [12]:
for i in tqdm(numerical_features):
    data_df['{}_diff'.format(i)] = data_df[i].diff()

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1252.78it/s]


In [13]:
data_df.fillna(method='bfill', inplace=True)

In [14]:
# 离散化
for f in numerical_features:
    data_df[f + '_20_bin'] = pd.cut(data_df[f], 20, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_50_bin'] = pd.cut(data_df[f], 50, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_100_bin'] = pd.cut(data_df[f], 100, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_200_bin'] = pd.cut(data_df[f], 200, duplicates='drop').apply(lambda x: x.left).astype(int)

In [15]:
for f1 in tqdm(['{}_20_bin'.format(i) for i in numerical_features] +
               ['{}_50_bin'.format(i) for i in numerical_features] +
               ['{}_100_bin'.format(i) for i in numerical_features] +
               ['{}_200_bin'.format(i) for i in numerical_features]):
    for f2 in numerical_features:
        tmp = data_df.groupby(f1, as_index=False)[f2].agg({
            '{}_{}_medi'.format(f1, f2): 'median',
            '{}_{}_mean'.format(f1, f2): 'mean',
            '{}_{}_max'.format(f1, f2): 'max',
            '{}_{}_min'.format(f1, f2): 'min',
            '{}_{}_sum'.format(f1, f2): 'sum',
            '{}_{}_std'.format(f1, f2): 'std',
            '{}_{}_skew'.format(f1, f2): 'skew'
        })
        data_df = data_df.merge(tmp, on=f1, how='left')
        del tmp
        gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:19<00:00,  1.01it/s]


In [16]:
train_df = data_df[:train_count].copy().reset_index(drop=True)
test_df = data_df[train_count:].copy().reset_index(drop=True)

del data_df
gc.collect()

20

In [17]:
train_time = train_df['time']
test_time = test_df['time']

In [18]:
drop_columns = ["time", "temperature"]

features = train_df[:1].drop(drop_columns, axis=1).columns
x_train = train_df[features]
x_test = test_df[features]

In [19]:
print(len(features))

734


In [20]:
psi_res = Parallel(n_jobs=4)(delayed(get_psi)(c) for c in tqdm(features))
psi_df = pd.concat(psi_res)

100%|███████████████████████████████████████████████████████████████████████████████| 734/734 [00:06<00:00, 108.33it/s]


In [21]:
features = list(psi_df[psi_df['PSI'] > 0.2]['变量名'].values)

In [22]:
print(features)

['day', 'outdoorTemp', 'outdoorAtmo', 'indoorAtmo', 'outdoorTemp_diff', 'outdoorHum_diff', 'outdoorAtmo_diff', 'indoorHum_diff', 'indoorAtmo_diff', 'outdoorTemp_20_bin', 'outdoorTemp_50_bin', 'outdoorTemp_100_bin', 'outdoorTemp_200_bin', 'outdoorAtmo_20_bin', 'outdoorAtmo_100_bin', 'outdoorAtmo_200_bin', 'indoorAtmo_100_bin', 'indoorAtmo_200_bin', 'outdoorTemp_20_bin_outdoorTemp_medi', 'outdoorTemp_20_bin_outdoorTemp_mean', 'outdoorTemp_20_bin_outdoorTemp_max', 'outdoorTemp_20_bin_outdoorTemp_min', 'outdoorTemp_20_bin_outdoorTemp_sum', 'outdoorTemp_20_bin_outdoorHum_medi', 'outdoorTemp_20_bin_outdoorHum_mean', 'outdoorTemp_20_bin_outdoorHum_max', 'outdoorTemp_20_bin_outdoorHum_min', 'outdoorTemp_20_bin_outdoorHum_sum', 'outdoorTemp_20_bin_outdoorHum_skew', 'outdoorTemp_20_bin_outdoorAtmo_medi', 'outdoorTemp_20_bin_outdoorAtmo_mean', 'outdoorTemp_20_bin_outdoorAtmo_max', 'outdoorTemp_20_bin_outdoorAtmo_min', 'outdoorTemp_20_bin_outdoorAtmo_sum', 'outdoorTemp_20_bin_indoorHum_medi', 'out

In [23]:
print(len(features))

298


In [24]:
x_train.drop(features, axis=1, inplace=True)
x_test.drop(features, axis=1, inplace=True)
gc.collect()

393

In [25]:
x_train.shape

(19338, 436)

In [26]:
x_train.head(10).append(x_train.tail(10))

,month,hour,min,outdoorHum,indoorHum,outdoorHum_20_bin,outdoorHum_50_bin,outdoorHum_100_bin,outdoorHum_200_bin,outdoorAtmo_50_bin,indoorHum_20_bin,indoorHum_50_bin,indoorHum_100_bin,indoorHum_200_bin,indoorAtmo_20_bin,indoorAtmo_50_bin,outdoorTemp_20_bin_outdoorTemp_std,outdoorTemp_20_bin_outdoorTemp_skew,outdoorTemp_20_bin_outdoorHum_std,outdoorTemp_20_bin_outdoorAtmo_std,outdoorTemp_20_bin_outdoorAtmo_skew,outdoorHum_20_bin_outdoorTemp_medi,outdoorHum_20_bin_outdoorTemp_mean,outdoorHum_20_bin_outdoorTemp_max,outdoorHum_20_bin_outdoorTemp_min,outdoorHum_20_bin_outdoorTemp_sum,outdoorHum_20_bin_outdoorTemp_std,outdoorHum_20_bin_outdoorTemp_skew,outdoorHum_20_bin_outdoorHum_medi,outdoorHum_20_bin_outdoorHum_mean,outdoorHum_20_bin_outdoorHum_max,outdoorHum_20_bin_outdoorHum_min,outdoorHum_20_bin_outdoorHum_sum,outdoorHum_20_bin_outdoorHum_std,outdoorHum_20_bin_outdoorHum_skew,outdoorHum_20_bin_outdoorAtmo_medi,outdoorHum_20_bin_outdoorAtmo_mean,outdoorHum_20_bin_outdoorAtmo_max,outdoorHum_20_bin_outdoorAtmo_min,outdoorHum_20_bin_outdoorAtmo_std,outdoorHum_20_bin_outdoorAtmo_skew,outdoorHum_20_bin_indoorHum_medi,outdoorHum_20_bin_indoorHum_mean,outdoorHum_20_bin_indoorHum_max,outdoorHum_20_bin_indoorHum_min,outdoorHum_20_bin_indoorHum_sum,outdoorHum_20_bin_indoorHum_std,outdoorHum_20_bin_indoorHum_skew,outdoorHum_20_bin_indoorAtmo_medi,outdoorHum_20_bin_indoorAtmo_mean,outdoorHum_20_bin_indoorAtmo_max,outdoorHum_20_bin_indoorAtmo_min,outdoorHum_20_bin_indoorAtmo_std,outdoorHum_20_bin_indoorAtmo_skew,outdoorAtmo_20_bin_outdoorTemp_medi,outdoorAtmo_20_bin_outdoorTemp_mean,outdoorAtmo_20_bin_outdoorTemp_min,outdoorAtmo_20_bin_outdoorTemp_std,outdoorAtmo_20_bin_outdoorAtmo_medi,outdoorAtmo_20_bin_outdoorAtmo_mean,outdoorAtmo_20_bin_outdoorAtmo_min,outdoorAtmo_20_bin_indoorAtmo_min,outdoorAtmo_20_bin_indoorAtmo_skew,indoorHum_20_bin_outdoorTemp_medi,indoorHum_20_bin_outdoorTemp_mean,indoorHum_20_bin_outdoorTemp_max,indoorHum_20_bin_outdoorTemp_min,indoorHum_20_bin_outdoorTemp_sum,indoorHum_20_bin_outdoorTemp_std,indoorHum_20_bin_outdoorTemp_skew,indoorHum_20_bin_outdoorHum_medi,indoorHum_20_bin_outdoorHum_mean,indoorHum_20_bin_outdoorHum_max,indoorHum_20_bin_outdoorHum_min,indoorHum_20_bin_outdoorHum_sum,indoorHum_20_bin_outdoorHum_std,indoorHum_20_bin_outdoorHum_skew,indoorHum_20_bin_outdoorAtmo_medi,indoorHum_20_bin_outdoorAtmo_mean,indoorHum_20_bin_outdoorAtmo_max,indoorHum_20_bin_outdoorAtmo_min,indoorHum_20_bin_outdoorAtmo_sum,indoorHum_20_bin_outdoorAtmo_std,indoorHum_20_bin_outdoorAtmo_skew,indoorHum_20_bin_indoorHum_medi,indoorHum_20_bin_indoorHum_mean,indoorHum_20_bin_indoorHum_max,indoorHum_20_bin_indoorHum_min,indoorHum_20_bin_indoorHum_sum,indoorHum_20_bin_indoorHum_std,indoorHum_20_bin_indoorHum_skew,indoorHum_20_bin_indoorAtmo_medi,indoorHum_20_bin_indoorAtmo_mean,indoorHum_20_bin_indoorAtmo_max,indoorHum_20_bin_indoorAtmo_min,indoorHum_20_bin_indoorAtmo_sum,indoorHum_20_bin_indoorAtmo_std,indoorHum_20_bin_indoorAtmo_skew,indoorAtmo_20_bin_outdoorTemp_medi,indoorAtmo_20_bin_outdoorTemp_mean,indoorAtmo_20_bin_outdoorTemp_max,indoorAtmo_20_bin_outdoorTemp_min,indoorAtmo_20_bin_outdoorTemp_sum,indoorAtmo_20_bin_outdoorTemp_std,indoorAtmo_20_bin_outdoorTemp_skew,indoorAtmo_20_bin_outdoorHum_mean,indoorAtmo_20_bin_outdoorHum_max,indoorAtmo_20_bin_outdoorHum_min,indoorAtmo_20_bin_outdoorHum_sum,indoorAtmo_20_bin_outdoorHum_std,indoorAtmo_20_bin_outdoorHum_skew,indoorAtmo_20_bin_outdoorAtmo_medi,indoorAtmo_20_bin_outdoorAtmo_max,indoorAtmo_20_bin_outdoorAtmo_min,indoorAtmo_20_bin_outdoorAtmo_sum,indoorAtmo_20_bin_outdoorAtmo_std,indoorAtmo_20_bin_outdoorAtmo_skew,indoorAtmo_20_bin_indoorHum_medi,indoorAtmo_20_bin_indoorHum_mean,indoorAtmo_20_bin_indoorHum_max,indoorAtmo_20_bin_indoorHum_min,indoorAtmo_20_bin_indoorHum_sum,indoorAtmo_20_bin_indoorHum_std,indoorAtmo_20_bin_indoorHum_skew,indoorAtmo_20_bin_indoorAtmo_medi,indoorAtmo_20_bin_indoorAtmo_mean,indoorAtmo_20_bin_indoorAtmo_max,indoorAtmo_20_bin_indoorAtmo_min,indoorAtmo

In [27]:
x_test.head(10).append(x_test.tail(10))

,month,hour,min,outdoorHum,indoorHum,outdoorHum_20_bin,outdoorHum_50_bin,outdoorHum_100_bin,outdoorHum_200_bin,outdoorAtmo_50_bin,indoorHum_20_bin,indoorHum_50_bin,indoorHum_100_bin,indoorHum_200_bin,indoorAtmo_20_bin,indoorAtmo_50_bin,outdoorTemp_20_bin_outdoorTemp_std,outdoorTemp_20_bin_outdoorTemp_skew,outdoorTemp_20_bin_outdoorHum_std,outdoorTemp_20_bin_outdoorAtmo_std,outdoorTemp_20_bin_outdoorAtmo_skew,outdoorHum_20_bin_outdoorTemp_medi,outdoorHum_20_bin_outdoorTemp_mean,outdoorHum_20_bin_outdoorTemp_max,outdoorHum_20_bin_outdoorTemp_min,outdoorHum_20_bin_outdoorTemp_sum,outdoorHum_20_bin_outdoorTemp_std,outdoorHum_20_bin_outdoorTemp_skew,outdoorHum_20_bin_outdoorHum_medi,outdoorHum_20_bin_outdoorHum_mean,outdoorHum_20_bin_outdoorHum_max,outdoorHum_20_bin_outdoorHum_min,outdoorHum_20_bin_outdoorHum_sum,outdoorHum_20_bin_outdoorHum_std,outdoorHum_20_bin_outdoorHum_skew,outdoorHum_20_bin_outdoorAtmo_medi,outdoorHum_20_bin_outdoorAtmo_mean,outdoorHum_20_bin_outdoorAtmo_max,outdoorHum_20_bin_outdoorAtmo_min,outdoorHum_20_bin_outdoorAtmo_std,outdoorHum_20_bin_outdoorAtmo_skew,outdoorHum_20_bin_indoorHum_medi,outdoorHum_20_bin_indoorHum_mean,outdoorHum_20_bin_indoorHum_max,outdoorHum_20_bin_indoorHum_min,outdoorHum_20_bin_indoorHum_sum,outdoorHum_20_bin_indoorHum_std,outdoorHum_20_bin_indoorHum_skew,outdoorHum_20_bin_indoorAtmo_medi,outdoorHum_20_bin_indoorAtmo_mean,outdoorHum_20_bin_indoorAtmo_max,outdoorHum_20_bin_indoorAtmo_min,outdoorHum_20_bin_indoorAtmo_std,outdoorHum_20_bin_indoorAtmo_skew,outdoorAtmo_20_bin_outdoorTemp_medi,outdoorAtmo_20_bin_outdoorTemp_mean,outdoorAtmo_20_bin_outdoorTemp_min,outdoorAtmo_20_bin_outdoorTemp_std,outdoorAtmo_20_bin_outdoorAtmo_medi,outdoorAtmo_20_bin_outdoorAtmo_mean,outdoorAtmo_20_bin_outdoorAtmo_min,outdoorAtmo_20_bin_indoorAtmo_min,outdoorAtmo_20_bin_indoorAtmo_skew,indoorHum_20_bin_outdoorTemp_medi,indoorHum_20_bin_outdoorTemp_mean,indoorHum_20_bin_outdoorTemp_max,indoorHum_20_bin_outdoorTemp_min,indoorHum_20_bin_outdoorTemp_sum,indoorHum_20_bin_outdoorTemp_std,indoorHum_20_bin_outdoorTemp_skew,indoorHum_20_bin_outdoorHum_medi,indoorHum_20_bin_outdoorHum_mean,indoorHum_20_bin_outdoorHum_max,indoorHum_20_bin_outdoorHum_min,indoorHum_20_bin_outdoorHum_sum,indoorHum_20_bin_outdoorHum_std,indoorHum_20_bin_outdoorHum_skew,indoorHum_20_bin_outdoorAtmo_medi,indoorHum_20_bin_outdoorAtmo_mean,indoorHum_20_bin_outdoorAtmo_max,indoorHum_20_bin_outdoorAtmo_min,indoorHum_20_bin_outdoorAtmo_sum,indoorHum_20_bin_outdoorAtmo_std,indoorHum_20_bin_outdoorAtmo_skew,indoorHum_20_bin_indoorHum_medi,indoorHum_20_bin_indoorHum_mean,indoorHum_20_bin_indoorHum_max,indoorHum_20_bin_indoorHum_min,indoorHum_20_bin_indoorHum_sum,indoorHum_20_bin_indoorHum_std,indoorHum_20_bin_indoorHum_skew,indoorHum_20_bin_indoorAtmo_medi,indoorHum_20_bin_indoorAtmo_mean,indoorHum_20_bin_indoorAtmo_max,indoorHum_20_bin_indoorAtmo_min,indoorHum_20_bin_indoorAtmo_sum,indoorHum_20_bin_indoorAtmo_std,indoorHum_20_bin_indoorAtmo_skew,indoorAtmo_20_bin_outdoorTemp_medi,indoorAtmo_20_bin_outdoorTemp_mean,indoorAtmo_20_bin_outdoorTemp_max,indoorAtmo_20_bin_outdoorTemp_min,indoorAtmo_20_bin_outdoorTemp_sum,indoorAtmo_20_bin_outdoorTemp_std,indoorAtmo_20_bin_outdoorTemp_skew,indoorAtmo_20_bin_outdoorHum_mean,indoorAtmo_20_bin_outdoorHum_max,indoorAtmo_20_bin_outdoorHum_min,indoorAtmo_20_bin_outdoorHum_sum,indoorAtmo_20_bin_outdoorHum_std,indoorAtmo_20_bin_outdoorHum_skew,indoorAtmo_20_bin_outdoorAtmo_medi,indoorAtmo_20_bin_outdoorAtmo_max,indoorAtmo_20_bin_outdoorAtmo_min,indoorAtmo_20_bin_outdoorAtmo_sum,indoorAtmo_20_bin_outdoorAtmo_std,indoorAtmo_20_bin_outdoorAtmo_skew,indoorAtmo_20_bin_indoorHum_medi,indoorAtmo_20_bin_indoorHum_mean,indoorAtmo_20_bin_indoorHum_max,indoorAtmo_20_bin_indoorHum_min,indoorAtmo_20_bin_indoorHum_sum,indoorAtmo_20_bin_indoorHum_std,indoorAtmo_20_bin_indoorHum_skew,indoorAtmo_20_bin_indoorAtmo_medi,indoorAtmo_20_bin_indoorAtmo_mean,indoorAtmo_20_bin_indoorAtmo_max,indoorAtmo_20_bin_indoorAtmo_min,indoorAtmo

In [28]:
col_corr = correlation(x_train, 0.98)
print(len(col_corr))

224


In [29]:
x_train.drop(list(col_corr), axis=1, inplace=True)
x_test.drop(list(col_corr), axis=1, inplace=True)

In [30]:
x_train['time'] = train_time
x_test['time'] = test_time

In [31]:
print(x_train.columns.to_list())

['month', 'hour', 'min', 'outdoorHum', 'outdoorAtmo_50_bin', 'indoorAtmo_20_bin', 'outdoorTemp_20_bin_outdoorTemp_std', 'outdoorTemp_20_bin_outdoorTemp_skew', 'outdoorTemp_20_bin_outdoorHum_std', 'outdoorTemp_20_bin_outdoorAtmo_std', 'outdoorTemp_20_bin_outdoorAtmo_skew', 'outdoorHum_20_bin_outdoorTemp_medi', 'outdoorHum_20_bin_outdoorTemp_min', 'outdoorHum_20_bin_outdoorTemp_sum', 'outdoorHum_20_bin_outdoorTemp_std', 'outdoorHum_20_bin_outdoorTemp_skew', 'outdoorHum_20_bin_outdoorHum_sum', 'outdoorHum_20_bin_outdoorHum_std', 'outdoorHum_20_bin_outdoorHum_skew', 'outdoorHum_20_bin_outdoorAtmo_medi', 'outdoorHum_20_bin_outdoorAtmo_mean', 'outdoorHum_20_bin_outdoorAtmo_max', 'outdoorHum_20_bin_outdoorAtmo_min', 'outdoorHum_20_bin_outdoorAtmo_std', 'outdoorHum_20_bin_outdoorAtmo_skew', 'outdoorHum_20_bin_indoorHum_max', 'outdoorHum_20_bin_indoorHum_std', 'outdoorHum_20_bin_indoorHum_skew', 'outdoorHum_20_bin_indoorAtmo_mean', 'outdoorHum_20_bin_indoorAtmo_max', 'outdoorHum_20_bin_indoorAt

In [32]:
to_drop = ['month', 'hour', 'min', 'outdoorHum']

In [33]:
x_train.drop(to_drop, axis=1, inplace=True)
x_test.drop(to_drop, axis=1, inplace=True)

In [34]:
data_df = pd.concat([x_train, x_test])
data_df.to_hdf('../../input/features/agg.h5', 'df')